In [2]:
import requests 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
df = pd.read_csv('URA_data.csv', index_col=0)
df = pd.get_dummies(df, columns = ['Type'], drop_first = True)
print(df.shape, '\n')
print(df.columns, '\n')
print(df.dtypes, '\n')

for col in df:
    print(df[col].value_counts())

(92622, 15) 

Index(['Project Name', 'Street Name', 'Postal District', 'Market Segment',
       'Tenure', 'Type of Sale', 'No. of Units', 'Price ($)', 'Nett Price ($)',
       'Area (Sqft)', 'Type of Area', 'Floor Level', 'Unit Price ($psf)',
       'Date of Sale', 'Type_Condominium'],
      dtype='object') 

Project Name         object
Street Name          object
Postal District       int64
Market Segment       object
Tenure               object
Type of Sale         object
No. of Units          int64
Price ($)             int64
Nett Price ($)       object
Area (Sqft)           int64
Type of Area         object
Floor Level          object
Unit Price ($psf)     int64
Date of Sale         object
Type_Condominium       bool
dtype: object 

Project Name
TREASURE AT TAMPINES     1715
PARC ESTA                1383
RIVERFRONT RESIDENCES    1370
STIRLING RESIDENCES      1202
JADESCAPE                1125
                         ... 
VILLA D'ESTE                1
THE SILVERTON               1


In [60]:
df['Lease_length'] = df['Tenure'].str.extract(r'([0-9]+) [Yy]rs')
df['Lease_start'] = df['Tenure'].str.extract(r'([0-9]+)[\s]*$')
df['Building_age'] = ((pd.to_datetime(df['Date of Sale'], format = '%b-%y')) - (pd.to_datetime(df['Lease_start']))) 

In [62]:
df['Lease_length'] = df['Lease_length'].astype(float)
df['Date of Sale'] = pd.to_datetime(df['Date of Sale'], format='%b-%y', errors='coerce')
df['Lease_start'] = pd.to_datetime(df['Lease_start'], errors='coerce')

# Calculate 'Remaining_tenure'
df['Remaining_tenure'] = df['Lease_length'].astype(float) - (
    (df['Date of Sale'] - df['Lease_start']) / np.timedelta64(1, 'D') / 365.25)

"""
    Read Me: Interpreting the relativity tenure, gives us a score of
    what the building's categorized tenure to be. For example, freehold and
    leasehold buildings will be placed at the exact uniform value of 1.0 (valtype=float)
"""
# Calculate the Relativity of the Tenure
df['Relative_tenure'] = 1.0 - 1.0 / ( ( 1.0 + df['Remaining_tenure'] / 208.96) ** 6.8828)
df.loc[df['Tenure'] == 'Freehold', 'Relative_tenure'] = 1.0

df

,Project Name,Street Name,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale,Type_Condominium,Lease_length,Lease_start,Building_age,Relative_tenure,Remaining_tenure
S/N,,,,,,,,,,,,,,,,,,,,
1,ONE SHENTON,SHENTON WAY,1,CCR,99 yrs lease commencing from 2005,Resale,1,2250000,-,1582,Strata,21 to 25,1422,2021-02-01,False,99.0,2005-01-01,5875 days,0.899753,82.915127
2,LANDRIDGE CONDOMINIUM,PASIR PANJANG ROAD,5,RCR,Freehold,Resale,1,2360000,-,1830,Strata,01 to 05,1290,2021-02-01,True,NaN,NaT,NaT,1.000000,NaN
3,THE ANCHORAGE,ALEXANDRA ROAD,3,RCR,Freehold,Resale,1,1860000,-,1378,Strata,01 to 05,1350,2021-02-01,True,NaN,NaT,NaT,1.000000,NaN
4,THE PARC CONDOMINIUM,WEST COAST WALK,5,OCR,Freehold,Resale,1,1800000,-,1927,Strata,01 to 05,934,2021-02-01,True,NaN,NaT,NaT,1.000000,NaN
5,COMMONWEALTH TOWERS,COMMONWEALTH AVENUE,3,RCR,99 yrs lease commencing from 2013,Resale,1,1225000,-,689,Strata,06 to 10,1778,2021-02-01,True,99.0,2013-01-01,2953 days,0.916777,90.915127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4663,SYMPHONY SUITES,YISHUN CLOSE,27,OCR,99 yrs lease commencing from 2014,New Sale,1,786000,-,797,Strata,01 to 05,987,2016-02-01,True,99.0,2014-01-01,761 days,0.927387,96.916496
4664,SYMPHONY SUITES,YISHUN CLOSE,27,OCR,99 yrs lease commencing from 2014,New Sale,1,973000,-,915,Strata,11 to 15,1063,2016-02-01,True,99.0,2014-01-01,761 days,0.927387,96.916496
4665,FLORAVILLE,CACTUS ROAD,28,OCR,Freehold,New Sale,1,763000,-,570,Strata,01 to 05,1337,2016-02-01,False,NaN,NaT,NaT,1.000000,NaN


In [68]:
df['Postal District'].value_counts()
df['PD_processed'] = df['Postal District'].apply(lambda x: 'other' if x in [6, 25, 26] else x)
df = pd.get_dummies(df, columns = ['PD_processed'], prefix = 'District')
df.drop('District_3', axis=1, inplace=True)

In [69]:
df = pd.get_dummies(df, columns=['Type of Sale'], prefix = 'SaleType', drop_first=True)

In [91]:
df['Floor_low'] = df['Floor Level'].str.extract(r'([B0-9]+) to')
df['Floor_high'] = df["Floor Level"].str.extract(r'to ([B0-9]+)')
#process any B1-B5 floor levels to an int
df['low_processed'] = df['Floor_low'].apply(lambda x: -1 if x == 'B1' else x)
df['high_processed'] = df['Floor_high'].apply(lambda x: -5 if x == "B5" else x)
df['Floor_number'] = df['high_processed'].astype(float) + df['low_processed'].astype(float) / 2.0
#df[floor_number]

df['dt_sale'] = pd.to_datetime(df['Date of Sale'], format='%b-%y')
df['Period'] = pd.PeriodIndex(df['dt_sale'], freq='Q') #quarterly analysis
df = pd.get_dummies(df, columns= ['Period'])

In [93]:
df['log_price_psf'] = np.log(df['Unit Price ($psf)'])
df['log_area_sqft'] = np.log(df['Area (Sqft)'])

target_columns = ['log_price_psf', 'log_area_sqft', 'Type_Condominium', 'Building_age', 'Relative_tenure', 'Floor_number'] + [c for c in df.columns if c.startswith('District_') or c.startswith('SaleType_') or c.startswith('Period_')]

df2 = df[target_columns]
df2

,log_price_psf,log_area_sqft,Type_Condominium,Building_age,Relative_tenure,Floor_number,District_1,District_1,District_1,District_2,...,Period_2020Q1,Period_2020Q1,Period_2020Q2,Period_2020Q2,Period_2020Q3,Period_2020Q3,Period_2020Q4,Period_2020Q4,Period_2021Q1,Period_2021Q1
S/N,,,,,,,,,,,,,,,,,,,,,
1,7.259820,7.366445,False,5875 days,0.899753,35.5,True,True,True,False,...,False,False,False,False,False,False,False,False,True,True
2,7.162397,7.512071,True,NaT,1.000000,5.5,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
3,7.207860,7.228388,True,NaT,1.000000,5.5,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,6.839476,7.563720,True,NaT,1.000000,5.5,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
5,7.483244,6.535241,True,2953 days,0.916777,13.0,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4663,6.894670,6.680855,True,761 days,0.927387,5.5,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4664,6.968850,6.818924,True,761 days,0.927387,20.5,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4665,7.198184,6.345636,False,NaT,1.000000,5.5,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [102]:
mean_building_age = df['Building_age'].mean()
mean_relative_tenure = df['Relative_tenure'].mean()
mean_Floor_number = df['Floor_number'].mean()
df2 = df2.fillna({'Building_age' : mean_building_age, 'Relative_tenure' : 
                mean_relative_tenure, 'Floor_number' : mean_Floor_number})
df2 #processed data frame, with no NAs, perhnaps
df2.isnull().sum()
df2.to_csv('ura_data_processed.csv')
print('Saved')

Saved
